# 层和块

首先，我们回顾一下多层感知机

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[ 2.4965e-01,  1.7340e-01, -2.8567e-01,  2.1272e-01, -2.7609e-04,
          9.4032e-02,  2.2910e-01,  8.8148e-02,  5.7029e-02,  1.4601e-01],
        [ 4.1981e-01,  1.7115e-01, -2.1900e-01,  1.0144e-01,  3.6402e-02,
          9.6631e-02,  2.0162e-01, -1.0708e-01,  9.1274e-03,  5.3098e-02]],
       grad_fn=<AddmmBackward>)

`nn.Sequential`定义了一种特殊的`Module`

自定义块

In [2]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)

    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

实例化多层感知机的层，然后在每次调用正向传播函数时调用这些层

In [3]:
net = MLP()
net(X)

tensor([[ 0.0451,  0.3717,  0.0621, -0.1061,  0.1060,  0.0670, -0.0715,  0.0492,
          0.2569,  0.2275],
        [-0.0420,  0.0906,  0.1692, -0.0599,  0.1329, -0.0098, -0.0157, -0.0128,
          0.2548,  0.0559]], grad_fn=<AddmmBackward>)

顺序块

In [5]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for block in args:
            self._modules[block] = block

    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.0367,  0.2082,  0.1047, -0.1591,  0.2000,  0.1441,  0.0279, -0.2801,
          0.2789, -0.3282],
        [ 0.1464,  0.1295,  0.1218, -0.2027,  0.1991,  0.1770,  0.0413, -0.0843,
          0.2275, -0.2201]], grad_fn=<AddmmBackward>)

在正向传播函数中执行代码

In [7]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

net = FixedHiddenMLP()
net(X)

tensor(0.0482, grad_fn=<SumBackward0>)

混合搭配各种组合块的方法

In [8]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(0.0733, grad_fn=<SumBackward0>)